In [1]:
#Instalando a versão mais nova do transformers direto da master
# !pip install git+https://github.com/huggingface/transformers
# !pip install datasets
!pip list | grep -E 'transformers|tokenizers|tensorflow'

tensorflow           2.0.0
tensorflow-estimator 2.0.0
tokenizers           0.10.2
transformers         4.6.0.dev0


In [2]:
import os

import pandas as pd

from datetime import datetime
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import BertProcessing
from tokenizers import ByteLevelBPETokenizer, BertWordPieceTokenizer
from transformers import ElectraTokenizerFast

In [3]:
DATA_DIR = 'data/'
TEXT_FILE = DATA_DIR + 'txt/text.txt'

if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR, mode=0o777)
    
if not os.path.exists(TEXT_FILE):
    print('Corpus not found!')

In [4]:
tokenizer = BertWordPieceTokenizer(
#     unk_token='[UNK]',
#     sep_token='[SEP]',
#     cls_token='[CLS]',
#     pad_token='[PAD]',
#     mask_token='[MASK]',
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=True,
    lowercase=True
)

tokenizer.pre_tokenizer = Whitespace()

In [ ]:
start_time = datetime.now()


tokenizer.train(
    files=[TEXT_FILE],
    vocab_size=52000,
    min_frequency=10,
    limit_alphabet=10000,
    special_tokens=['[UNK]', '[CLS]', '[SEP]', '[PAD]', '[MASK]'],
    wordpieces_prefix='##',
    show_progress=True
)

print('Duration: {}'.format(datetime.now() - start_time))

In [ ]:
VOCAB_DIR = DATA_DIR + 'vocab/'

if not os.path.exists(VOCAB_DIR):
  os.makedirs(VOCAB_DIR, mode=0o777)

tokenizer.save_model(VOCAB_DIR)

In [ ]:
tokenizer.token_to_id('[SEP]')

In [ ]:
sentence = 'Sou da paz!'

In [ ]:
saida = tokenizer.encode(sentence)

In [ ]:
print(saida.tokens)
print(saida.ids)

In [ ]:
print(saida.offsets[3])
print(sentence[10:11])

In [ ]:
sentence = [['Sou da paz!', 'Mas não mexa comigo!'], ['Um novo treinamento a cada dia.', 'Só assim se tem um campeão.']]

In [ ]:
saida = tokenizer.encode_batch(sentence)

In [ ]:
tokenizer.enable_padding(pad_id=3, pad_token='[PAD]')

In [ ]:
print(saida[1].tokens)

In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ('[SEP]', tokenizer.token_to_id('[SEP]')),
    ('[CLS]', tokenizer.token_to_id('[CLS]')),
)

tokenizer.enable_truncation(max_length=512)

In [ ]:
tokenizer.encode('Sou da paz!').tokens